## Part 1

In [8]:
mutable struct Element
    name::Symbol
    quant::Int
    formula::Dict{Symbol, Int}
    required::Int
    residue::Int
    used_in::Set{Symbol}
    is_ready::Bool
end
Element(name::Symbol) = Element(name, 0, Dict{Symbol, Int}(), 0, 0, Set([]), false)
sym(s) = Symbol(strip(s))
quant(s) = parse(Int, strip(s))

struct NanoFactory
    elements::Dict{Symbol, Element}
    ore_cnt::Vector{Int}
end

function NanoFactory(desc::Vector{T}) where T
    elements = Dict{Symbol, Element}()
    for s in desc add_element!(elements, s) end
    NanoFactory(elements, Int[])
end

function add_element!(elements, s)
    m = match(r"^(.*) => ([0-9]+) (.*)$", s)
    product = get!(elements, sym(m[3]), Element(sym(m[3])))
    product.quant = quant(m[2])
    m = split(m[1], ',')
    for elem_str in m
        term = match(r"([0-9]+) (.*)$", elem_str)
        elem = get!(elements, sym(term[2]), Element(sym(term[2])))
        product.formula[elem.name] = quant(term[1])
        push!(elem.used_in, product.name)
    end
end

add_element! (generic function with 2 methods)

In [15]:
function process_elem!(f::NanoFactory, start=:FUEL)
    stack = Vector{Symbol}()
    f.elements[start].required = 1
    f.elements[start].is_ready = true
    f.elements[start].residue = 0
    push!(stack, start)
    while !isempty(stack)
        elem = f.elements[popfirst!(stack)]
        elem.is_ready = elem.is_ready || all([f.elements[x].is_ready for x in elem.used_in])
        if elem.is_ready
            if elem.quant > 0
                multiplier = div(elem.required - elem.residue, elem.quant) + 
                    Int(mod(elem.required - elem.residue, elem.quant) != 0)
                elem.residue = multiplier*elem.quant + elem.residue - elem.required
                for (x, v) in elem.formula
                    if !(x in stack) push!(stack, x) end
                    f.elements[x].required += multiplier * v
                end
            end
        else
            push!(stack, elem.name)
        end
    end
end

function restart!(f::NanoFactory)
    for elem in values(f.elements)
        elem.is_ready = false
        elem.required = 0
    end
end

function update_ore!(f::NanoFactory, elem = :FUEL)
    restart!(f)
    process_elem!(f, elem)
    total_ore = isempty(f.ore_cnt) ? 0 : f.ore_cnt[end]
    push!(f.ore_cnt, f.elements[:ORE].required + total_ore)
end

update_ore! (generic function with 2 methods)

In [13]:
# 2210736
nanofactory = NanoFactory(readlines("test5.txt"))
update_ore!(nanofactory)
println(nanofactory.ore_cnt[end])

2210736


In [14]:
# 443537
nanofactory = NanoFactory(readlines("input.txt"))
update_ore!(nanofactory)
println("Part 1: ", nanofactory.ore_cnt[end])

Part 1: 443537


## Part 2

In [39]:
nanofactory = NanoFactory(readlines("test3.txt"))
update_ore!(nanofactory)
println(nanofactory.ore_cnt[end])

13312


In [54]:
function generate_cycle(f::NanoFactory, lim)
    update_ore!(f)
    while any([elem.residue != 0 for elem in values(f.elements)]) && f.ore_cnt[end] < lim
        update_ore!(f)
    end
end

function calc_fuel(f::NanoFactory, ore)
    generate_cycle(f, ore)
    fuel = div(ore, f.ore_cnt[end])*length(f.ore_cnt)
    residual_ore = mod(ore, f.ore_cnt[end])
    fuel += sum(f.ore_cnt .<= residual_ore)
    
    fuel
end

calc_fuel (generic function with 1 method)

In [56]:
nanofactory = NanoFactory(readlines("input.txt"))
calc_fuel(nanofactory, 1000000000000)

2910558

In [57]:
nanofactory.ore_cnt

2910559-element Array{Int64,1}:
        443537
        858403
       1109059
       1523115
       1772062
       2180294
       2450835
       2866012
       3119181
       3524148
       3944020
       4188979
       4626693
             ⋮
  999996246991
  999996498543
  999996904960
  999997161174
  999997567855
  999997841190
  999998256268
  999998502865
  999998913541
  999999331439
  999999577919
 1000000009891

In [52]:
1000000000000/416755326160

2.3994894299589147

In [27]:
update_ore!(nanofactory)
println(nanofactory.ore_cnt[end])

145


In [28]:
for v in values(nanofactory.elements)
    println(v.name, ": ", v.residue)
end

FUEL: 0
A: 0
D: 0
ORE: 0
B: 0
E: 0
C: 0


In [30]:
nanofactory.ore_cnt

5-element Array{Int64,1}:
  31
  62
  93
 124
 145

In [25]:
1000000000000

1000000000000

In [26]:
180697*5586022

1009377417334

In [27]:
13312*82892753

1103468327936